In [2]:
import numpy as np
import pandas as pd
import os
import sys as sys
import socket
import matplotlib.pyplot as plt
import multiprocessing as mp
import itertools as it
#from adjustText import adjust_text

import warnings
warnings.filterwarnings("ignore") # Great Style
socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("No compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

#sys.path.insert(0,"/n/groups/reich/hringbauer/git/hapBLOCK/package/")  # hack to get development package first in path
from ancIBD.ibd_stats.funcs import new_columns, find_relatives, give_sub_df, plot_age_diff, rc_date

compute-a-17-57.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 32


# 1) Browse full IBD dataset (Eurasia)

In [8]:
#path_save = f"/n/groups/reich/hringbauer/git/ibd_euro/output/ibd/ibd220.ind.v56.3.tsv" # for IBD csv file
#path_meta_ibd =  f"/n/groups/reich/hringbauer/git/ibd_euro/data/iid_lists/iid_ibd_v56.3.tsv" # for iids

path_save = f"/n/groups/reich/hringbauer/git/ibd_euro/output/ibd/ibd220.ind.v61.0a.tsv" # for IBD csv file
path_meta_ibd =  f"/n/groups/reich/hringbauer/git/ibd_euro/data/iid_lists/iid_ibd_v61.0.tsv" # for iids

df = pd.read_csv(path_save, sep="\t")
df_meta_ibd =  pd.read_csv(path_meta_ibd, sep="\t")

new_columns(df, df_meta_ibd, col = ["age", "loc", "n_cov_snp", "frac_gp", "clst"]) #"clst"

df=df.sort_values(by="sum_IBD>20", ascending=False)
df20 = df[df["max_IBD"]>16].copy()

In [ ]:
give_sub_df(df, pop1='', pop2='I24494', col='iid').sort_values(by="max_IBD", ascending=False)[:50]

# Cut out IBD within Punic

In [23]:
df_p = pd.read_csv("/n/groups/reich/hringbauer/git/punic_aDNA/data/cluster_assignments_punic.v54.1j.tsv", sep="\t")
iids = df_p["iid"].values

idx = df_meta_ibd["iid"].isin(iids)
print(f"Found {np.sum(idx)} Punic IIDs in IBD Dataset")
dft = df_meta_ibd[idx]

s = "|".join(iids)

Found 82 Punic IIDs in IBD Dataset


In [28]:
idx = (df["iid1"].isin(iids)) & (df["iid2"].isin(iids))
print(f"Found {np.sum(idx)}/{len(idx)} Punic IBD pairs")
df_s = df[idx].copy()

Found 95/753387 Punic IBD pairs


In [29]:
df_s

,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2,clst1,clst2
426,I22122,I22118,213.983188,3310.906489,25.0,3310.906489,25.0,3310.906489,25.0,3276.589797,23.0,2419.0,2589.0,"Sardinia, Tharros","Sardinia, Tharros",711166,802359,Italy_Sardinia_Punic_Early,Italy_Sardinia_Punic_Early
989,I24494,I24193,283.652203,2005.149298,32.0,1985.909503,30.0,1958.025506,28.0,1922.099313,26.0,2464.0,2500.0,Kerkouane,Kerkouane,791516,789887,Tunisia_Punic_mother.or.daughter.I24040,Tunisia_Punic
1243,I22122,I22117,176.443803,1689.705997,23.0,1689.705997,23.0,1689.705997,23.0,1689.705997,23.0,2419.0,2582.0,"Sardinia, Tharros","Sardinia, Tharros",711166,791179,Italy_Sardinia_Punic_Early,Italy_Sardinia_Punic_Early
2162,I22001,I22000,95.567897,917.425610,22.0,906.871714,21.0,894.147016,20.0,856.561117,18.0,2500.0,2500.0,"Sicily, Marsala, Lilybaeum, Necropoli Monumentale","Sicily, Marsala, Lilybaeum, Necropoli Monumentale",743817,803880,Italy_Sicily_Punic_Possible,Italy_Sicily_Punic_Possible
2611,I24194,I24494,75.874001,714.631484,16.0,696.773581,14.0,696.773581,14.0,679.805879,13.0,2500.0,2464.0,Kerkouane,Kerkouane,791265,791516,Tunisia_Punic,Tunisia_Punic_mother.or.daughter.I24040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262694,I24196,I35339,10.757494,10.757494,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2500.0,2500.0,Kerkouane,"Tunis, Carthage",553006,759862,Tunisia_Punic,Tunisia_Punic
276883,I35339,I4799,10.506599,10.506599,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2500.0,2442.0,"Tunis, Carthage","Sicily, Motya",759862,837043,Tunisia_Punic,Italy_Sicily_Punic_Early
277318,I24032,I22235,10.499001,10.499001,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2500.0,2620.0,Kerkouane,"Sicily, Motya",758572,749463,Tunisia_Punic,Italy_Sicily_Punic_Early
273565,I27613,I28504,10.559803,10.559803,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2225.0,2337.0,"Eivissa, Ses Païsses de Cala d'Hort",Carthage,681319,679772,Spain_Punic,Tunisia_Punic


### Save IBD within Punic data [for Supp. Table!]

In [27]:
dft.to_csv("/n/groups/reich/hringbauer/git/punic_aDNA/output/tables/ibd/iids_ibd_punic.v56.3.tsv", sep="\t", index=False)
df_s.to_csv("/n/groups/reich/hringbauer/git/punic_aDNA/output/tables/ibd/ibd_punic.v56.3.tsv", sep="\t", index=False)

# 2) Browse specfically run Punic IBD Dataset
Contains more low coverage Indidivuals. Only run within Punic data!

In [ ]:
df = pd.read_csv("/n/groups/reich/hringbauer/git/punic_aDNA/output/ibd/v54.1.ibd_ind.d220.tsv", sep="\t")
path_save = "/n/groups/reich/hringbauer/Data/v54.1.anno.csv"# for IBD csv file
df_meta = pd.read_csv(path_save, sep=",")

new_columns(df, df_meta, col = ["age", "loc", "n_cov_snp"]) #"clst"

In [5]:
iids = ["18199", "I18201", "I18203"]


In [ ]:
df[df["iid1"].isin(iids) | df["iid2"].isin(iids)]

# Area 51

### Check Intersection v61.0 IBD run & Punic data

In [7]:
df_p = pd.read_csv("/n/groups/reich/hringbauer/git/punic_aDNA/data/cluster_assignments_punic.v54.1j.tsv", sep="\t")
df_iid = pd.read_csv("/n/groups/reich/hringbauer/git/ibd_euro/data/iid_lists/iid_ibd_v61.0.tsv", sep="\t") # for iids

In [8]:
len(df_iid)

16287

In [10]:
dft = pd.merge(df_p, df_iid, left_on="iid", right_on="Master ID")
print(f"Found {len(dft)} of {len(df_p)} Punic IIDs in IBD v61.0 run iids")

Found 82 of 160 Punic IIDs in IBD v61.0 run iids


In [11]:
dft["location"].value_counts()

location
Lilybaeum     14
Kerkouene     13
Birgi         12
Tharros       11
Motya          6
Carthage       6
Eivissa        5
Malaga         4
Selinunte      4
Villaricos     3
Palermo        2
Cadiz          1
Akhziv         1
Name: count, dtype: int64